In [48]:
import sys
import os  
from datetime import datetime, timedelta, date
import datetime as dt

import requests
import pandas as pd
import json
import yaml
import importlib


from snowflake.snowpark.functions import date_trunc, current_date, to_date
from snowflake.snowpark.functions import col, lit, when
from snowflake.snowpark.types import DateType
import snowflake.snowpark.functions as f


sys.path.append(os.path.abspath('scripts'))

try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
except Exception:
    import SnowflakeConnector  # brings module into scope
    importlib.reload(SnowflakeConnector)
    from SnowflakeConnector import create_active_session
    session = create_active_session()


import utils
importlib.reload(utils)
from utils import get_orders_for_date, backfill_date_generator, get_min_date_from_table



In [46]:
#Access Token
TOKEN_PATH = "config/shopify_auth.yaml"
QUERY_PATH = "graphql/queries/get_transactions.graphql"
SELLING_PLAN_TABLE = "snowpark_db.shopify.orders_with_selling_plan"
DATE_COLUMN = "PROCESSED_AT"


with open(TOKEN_PATH, "r") as file:
    config = yaml.safe_load(file)
ACCESS_TOKEN = config.get("ACCESS_TOKEN", {})


with open(QUERY_PATH, "r") as f:
    QUERY_STRING = f.read()


SHOP_ENDPOINT = "https://shopmbg.myshopify.com/admin/api/2024-04/graphql.json"

In [ ]:
orders = get_orders_for_date("2025-06-11", ACCESS_TOKEN, SHOP_ENDPOINT, QUERY_STRING)




In [5]:
#get_orders_for_date

# def get_orders_for_date(date, ACCESS_TOKEN=ACCESS_TOKEN):

#     headers = {
#     "Content-Type": "application/json",
#     "X-Shopify-Access-Token": ACCESS_TOKEN
#     }

#     url = "https://shopmbg.myshopify.com/admin/api/2024-04/graphql.json"
#     all_orders = []
#     cursor = None

#     start = datetime.strptime(date, "%Y-%m-%d")
#     end = start + timedelta(days=1)
#     #date_query_string = f"processed_at:>={start.strftime('%Y-%m-%dT00:00:00Z')} AND processed_at:<{end.strftime('%Y-%m-%dT00:00:00Z')}"

#     date_query_string  = f"processed_at:={date}"

#     while True:
#         variables = {
#             "cursor": cursor,
#             "date": date_query_string
#         }
#         payload = {
#             "query": query_str,
#             "variables": variables
#         }

#         # dbug print(payload)
#         response = requests.post(url, json=payload, headers=headers)
#         response.raise_for_status()  # raise if bad response
#         data = response.json()

#         # Defensive: check for errors
#         if "errors" in data:
#             raise Exception(f"GraphQL errors: {data['errors']}")

#         orders_data = data["data"]["orders"]
#         edges = orders_data["edges"]

#         # Append current page orders
#         all_orders.extend(edges)

#         page_info = orders_data["pageInfo"]
#         if not page_info["hasNextPage"]:
#             break
#         cursor = page_info["endCursor"]

#     print(f"Fetched {len(all_orders)} orders for {date}")
   

#     return all_orders  

In [51]:
#Function usage example

get_date = backfill_date_generator(start_date=date(2025, 6, 20), stop_date=date(2025, 6, 15))
for _ in range(10):
    print(next(get_date))

2025-06-19
2025-06-18
2025-06-17
2025-06-16
2025-06-15
Reached stop date: 2025-06-15. Generator exhausted.


StopIteration: 

In [ ]:
#date_list = date_generator(stop_date=date(2025, 1, 1))


date_list = ["2025-06-17", "2025-06-18"]

for next_date in date_list:
    all_orders = get_orders_for_date(next_date)
    now = datetime.utcnow()
    paginated_df = pd.DataFrame([
        {
            "PROCESSED_AT" :o['node']['processedAt'] ,
            "ORDER_ID": o['node']['id'].split('/')[-1],
            "ORDER_NAME": o['node']['name'].split('#')[-1],
            "LINEITEM_ID":json.dumps(o['node']['lineItems']),
            "TAGS":json.dumps(o['node']['tags']),
            "LOAD_TIMESTAMP": now
        }
        for o in all_orders
    ])

    #
    end = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)
    end = end.strftime('%Y-%m-%dT00:00:00Z')
    paginated_df = paginated_df[paginated_df["PROCESSED_AT"]<end]
    

    records = paginated_df.to_dict(orient="records")
    transactions = session.create_dataframe(records)
    transactions.write.save_as_table("snowpark_db.shopify.orders_with_selling_plan", mode="append")
 

    print(f"Updated transactions for {next_date}")

In [32]:

date = '2025-06-16'
all_orders = get_orders_for_date(date)
now = datetime.utcnow()
paginated_df = pd.DataFrame([
        {
            "PROCESSED_AT" :o['node']['processedAt'] ,
            "ORDER_ID": o['node']['id'].split('/')[-1],
            "ORDER_NAME": o['node']['name'].split('#')[-1],
            "LINEITEM_ID":json.dumps(o['node']['lineItems']),
            "TAGS":json.dumps(o['node']['tags']),
            "LOAD_TIMESTAMP": now
        }
        for o in all_orders
    ])

end = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)
end = end.strftime('%Y-%m-%dT00:00:00Z')
paginated_df = paginated_df[paginated_df["PROCESSED_AT"]<end]


print(f"end = {end}")




paginated_df.tail(10)

#1769

Fetched 909 orders for 2025-06-16
end = 2025-06-17T00:00:00Z


,PROCESSED_AT,ORDER_ID,ORDER_NAME,LINEITEM_ID,TAGS,LOAD_TIMESTAMP
854,2025-06-16T23:22:59Z,6685462102121,864481,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""CashBack:25%"", ""nofraud_pass"", ""Subscription...",2025-06-18 18:41:07.307691
855,2025-06-16T23:27:02Z,6685467279465,864482,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""CashBack:25%"", ""nofraud_pass"", ""Subscription...",2025-06-18 18:41:07.307691
856,2025-06-16T23:27:29Z,6685467967593,864483,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass"", ""Subscription First Order""]",2025-06-18 18:41:07.307691
857,2025-06-16T23:41:23Z,6685484122217,864484,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""CashBack:25%"", ""nofraud_pass""]",2025-06-18 18:41:07.307691
858,2025-06-16T23:45:16Z,6685489627241,864485,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass"", ""Subscription First Order""]",2025-06-18 18:41:07.307691
859,2025-06-16T23:48:18Z,6685493428329,864486,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""CashBack:25%"", ""nofraud_pass""]",2025-06-18 18:41:07.307691
860,2025-06-16T23:51:57Z,6685497655401,864487,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass"", ""Subscription First Order""]",2025-06-18 18:41:07.307691
861,2025-06-16T23:57:25Z,6685505028201,864488,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass""]",2025-06-18 18:41:07.307691
862,2025-06-16T23:58:11Z,6685505814633,864489,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass""]",2025-06-18 18:41:07.307691
863,2025-06-16T23:59:30Z,6685507649641,864490,"{""edges"": [{""node"": {""id"": ""gid://shopify/Line...","[""nofraud_pass""]",2025-06-18 18:41:07.307691
